In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [57]:
train = pd.read_csv('src/X_train.csv')
test = pd.read_csv('src/X_test.csv')

In [58]:
#ID variable
train_ID = train['Id']
test_ID = test['Id']

In [59]:
#drop the  'Id' colum 
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [60]:
y_train=train["SalePrice"]
#drop the  target colum 
train.drop("SalePrice", axis = 1, inplace = True)

In [61]:
gbm = GradientBoostingRegressor(random_state=36)
rf = RandomForestRegressor(random_state = 36)

## Setup of the Random Forest Randomized Hyperparameter Search

In [62]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 5, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [63]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               scoring="neg_mean_squared_error", cv = 4, verbose=2, random_state=36, n_jobs = -1)

In [64]:
# Fit the random search model
rf_random.fit(train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 11.5min finished


RandomizedSearchCV(cv=4, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=36,
           verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'min_samples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [2, 5, 10], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=36, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=2)

In [65]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=40,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1400, n_jobs=1, oob_score=False, random_state=36,
           verbose=0, warm_start=False)

In [66]:
rf_random.best_score_

-0.019476877270038967

In [69]:
y_test=rf_random.predict(test)

Reverting ln(x+1) transformation on target

In [79]:
y_test = np.expm1(y_test)

In [80]:
df = pd.DataFrame({'Id':test_ID, 'SalePrice':y_test})

In [81]:
df.to_csv("output/submission.csv", index=False)